<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/both-logos-small-alpha.png" style="display:inline"></span><br/>

# tâches, valeurs de retour, exceptions

# `ensure_future()`

 retourne un objet `Task`
* qui connaît l'état (terminé ?)
* et le résultat de l'exécution

[animation](loop-stacks/index.html)

# bonnes pratiques

* attraper les exceptions
  * dans le code asynchrone

* autrement
  * lire les exceptions (appeler `Task.exception()`)

# lire les tâches

* méthode de classe `Task.all_tasks(`*loop=None*`)` 
* méthode de classe `Task.current_task(`*loop=None*`)` 

In [ ]:
from asyncio import (
    get_event_loop, ensure_future, Task )

In [ ]:
async def foo():
    pass


In [ ]:
task = ensure_future(foo())
task

In [ ]:
# attention : retourne un set
tasks = Task.all_tasks()
tasks

In [ ]:
task in tasks

In [ ]:
# et comme on n'est pas dans la  boucle
print(task.current_task())

# `task.cancel()`

pour annuler la tâche:

* continue de donner la main à la tâche
* mais envoie une exception `CancelledError` au prochain tour

# résumé

* une boucle = ensemble de tâches (de *futures*)

* une tâche
  * une pile
  * un état (terminé ?)
  * un résultat ou une exception

* responsabilité de l'appelant
  * de lire les résultats et exceptions
  * sinon : avertissement émis par le gc